<a href="https://colab.research.google.com/github/d9249/DACON/blob/main/CVLC_06_DenseNet169.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/DACON_CVLC/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/DACON_CVLC/data/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/DACON_CVLC/data/submission.csv')

In [ ]:
# !mkdir /content/drive/MyDrive/DACON_CVLC/data/images_train
# !mkdir /content/drive/MyDrive/DACON_CVLC/data/images_train/0
# !mkdir /content/drive/MyDrive/DACON_CVLC/data/images_train/1
# !mkdir /content/drive/MyDrive/DACON_CVLC/data/images_train/2
# !mkdir /content/drive/MyDrive/DACON_CVLC/data/images_train/3
# !mkdir /content/drive/MyDrive/DACON_CVLC/data/images_train/4
# !mkdir /content/drive/MyDrive/DACON_CVLC/data/images_train/5
# !mkdir /content/drive/MyDrive/DACON_CVLC/data/images_train/6
# !mkdir /content/drive/MyDrive/DACON_CVLC/data/images_train/7
# !mkdir /content/drive/MyDrive/DACON_CVLC/data/images_train/8
# !mkdir /content/drive/MyDrive/DACON_CVLC/data/images_train/9
# !mkdir /content/drive/MyDrive/DACON_CVLC/data/images_test

In [ ]:
# import cv2

# for idx in range(len(train)) :
#     img = train.loc[idx, '0':].values.reshape(28, 28).astype(int)
#     digit = train.loc[idx, 'digit']
#     cv2.imwrite(f'/content/drive/MyDrive/DACON_CVLC/data/images_train/{digit}/{train["id"][idx]}.png', img)

# for idx in range(len(test)) :
#     img = test.loc[idx, '0':].values.reshape(28, 28).astype(int)
#     cv2.imwrite(f'/content/drive/MyDrive/DACON_CVLC/data/images_test/{test["id"][idx]}.png', img)

In [ ]:
import tensorflow as tf
model =  tf.keras.applications.densenet.DenseNet169(weights=None, include_top=True, input_shape=(224, 224, 1), classes=10)

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.002,epsilon=None), metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2,
                             rotation_range=10,
                             width_shift_range=0.1,
                             height_shift_range=0.1)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/DACON_CVLC/data/images_train', target_size=(224,224), color_mode='grayscale', class_mode='categorical', subset='training')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/DACON_CVLC/data/images_train', target_size=(224,224), color_mode='grayscale', class_mode='categorical', subset='validation')

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'/content/drive/MyDrive/DACON_CVLC/Checkpoint/CVLC_06_DenseNet169.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

In [ ]:
model.fit_generator(train_generator, epochs=500, validation_data=val_generator, callbacks=[checkpoint])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(model.history.history["accuracy"], label='accuracy')
plt.plot(model.history.history["val_accuracy"], label='valid_accuracy')

plt.legend()
plt.show()

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/DACON_CVLC/Checkpoint/CVLC_06_DenseNet169.h5', compile=False)

In [ ]:
!mkdir /content/drive/MyDrive/DACON_CVLC/data/images_test/none
!mv /content/drive/MyDrive/DACON_CVLC/data/images_test/*.png /content/drive/MyDrive/DACON_CVLC/data/images_test/none

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
test_generator = datagen.flow_from_directory('/content/drive/MyDrive/DACON_CVLC/data/images_test', target_size=(224,224), color_mode='grayscale', class_mode='categorical', shuffle=False)

In [ ]:
# test_generator.reset()
# predict = model.predict_generator(test_generator).argmax(axis=1)

In [ ]:
# submission = pd.read_csv('/content/drive/MyDrive/DACON_CVLC/data/submission.csv')
# submission['digit'] = predict
# submission.to_csv('/content/drive/MyDrive/DACON_CVLC/Submission/submission.csv', index=False)

In [ ]:
sub['digit'] = np.argmax(model.predict(test_generator), axis=1)
sub.head()
sub.to_csv('/content/drive/MyDrive/DACON_CVLC/Submission/CVLC_06_DenseNet169.csv', index=False)